In [154]:
username = "Alex"
env = "integrate"
pathtorepo = "C:\\Users\\" *username *  "\\Desktop\\"
using Pkg
Pkg.activate(pathtorepo * "dynamical-systems\\env\\" * env * "\\")
include(pathtorepo * "dynamical-systems\\system.jl")

using StaticArrays, DifferentialEquations, DynamicalSystems
sE, sI, rE, rI, Y  = -50..50, -50..50,  -50..50, -50..50, -50..50
box = sE × sI × rE × rI × Y
using GLMakie, LinearAlgebra

  Activating project at `C:\Users\Alex\Desktop\dynamical-systems\env\integrate`


In [155]:
function get_valus_vectors(fp)
    Jfp = rate_jac(fp, p, 0.0)
    Jfp_value_vectors = eigen(Jfp)
    Jfp_value = Jfp_value_vectors.values
    Jfp_vectors = Jfp_value_vectors.vectors
    return Jfp_value, Jfp_vectors
end

get_valus_vectors (generic function with 1 method)

In [156]:
τsE = 3.0; γE = 4.0; s0E = 0.15;
τsI = 10.0; γI = 8.0; s0I = 0.32507438 # 0.3267
# IE = 0.9; 
τrE = 2.0; kE = 5.0; IE = 0.9; wEE = 3.5; wIE = 5.0; θE = 0.2;
τrI = 6.0; kI = 5.0; II = 0.0; wEI = 5.0; wII = 3.0; θI = 0.4;

τY = 0.01;  βY = 0.0;
ythr = 0.5; sEthr = 0.5; kY = 0.01; γY = 0.0;

In [157]:
time = 1000; tt = 0.0; tstep = 0.001; times = [time, tt]
integ_set = (alg = Vern9(), adaptive = false, dt = tstep);

In [158]:
p = [τsE, γE, s0E, τsI, γI, s0I, τrE, kE, IE, wEE, wIE, θE, τrI, kI, II, wEI, wII, θI, τY, βY, γY, ythr, sEthr, kY];
u0 = [0.0, 0.0, 0.0, 0.0, 0.0]

ds = CoupledODEs(model_braint_rhythms, u0, p, diffeq = integ_set);

In [159]:
fp, _, _ = fixedpoints(ds, box, rate_jac);

In [162]:
fp[1]

5-element SVector{5, Float64} with indices SOneTo(5):
 0.2038456920320365
 0.44520847844293177
 0.016908057738664897
 0.02706739689031421
 2.74791723210987e-82

In [160]:
valuesfp1, vectorsfp1 = get_valus_vectors(fp[1])

(ComplexF64[-100.0 + 0.0im, -0.7966545770528577 + 0.0im, -0.34769198084889685 + 0.0im, 7.428170230836667e-5 - 0.2073332995889558im, 7.428170230836667e-5 + 0.2073332995889558im], ComplexF64[0.0 + 0.0im -0.9089195423214191 + 0.0im … 0.7074458324564599 - 0.0im 0.7074458324564599 + 0.0im; 0.0 + 0.0im -0.0974013243370146 + 0.0im … 0.06215817149922635 + 0.5692661186299908im 0.06215817149922635 - 0.5692661186299908im; … ; 0.0 + 0.0im 0.14813212624242167 + 0.0im … 0.2829761724060193 + 0.1270934182245284im 0.2829761724060193 - 0.1270934182245284im; 1.0 + 0.0im 0.0 + 0.0im … 0.0 - 0.0im 0.0 + 0.0im])

In [161]:
valuesfp1

5-element SVector{5, ComplexF64} with indices SOneTo(5):
               -100.0 + 0.0im
  -0.7966545770528577 + 0.0im
 -0.34769198084889685 + 0.0im
 7.428170230836667e-5 - 0.2073332995889558im
 7.428170230836667e-5 + 0.2073332995889558im